<a href="https://colab.research.google.com/github/ccnmagnoo/colabs/blob/main/sec_etl_professional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Resume
Actually there´s 11.700 profesional certificated in GAS and Electricity procedures. But Just 290 of them are women, and [SEC](www.sec.cl) (institution focused in regulate this profesionals), dosen´t focus in this labor. This code is used for extract and filter those women to use in diverse social programs, and visualice their labor and effort in this energy field

In [128]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [129]:
import pandas as pd
SOURCE_WEB = 'https://wlhttp.sec.cl/buscadorinstaladores/buscador.do'
PATH_GDRIVE = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality'
eli_source = f'{PATH_GDRIVE}/sec_eli.xls'
gas_source = f'{PATH_GDRIVE}/sec_gas.xls'

#Preparing Dataset

In [130]:
#read gas & dataframe it
gas_df = pd.read_excel(gas_source,header=4)
gas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              1898 non-null   object
 1   CORREO ELECTRÓNICO  1867 non-null   object
 2   RUT                 1898 non-null   object
 3   DIRECCIÓN           1898 non-null   object
 4   TELÉFONO            902 non-null    object
 5   CELULAR             1844 non-null   object
 6   LICENCIA CLASE      1898 non-null   object
dtypes: object(7)
memory usage: 103.9+ KB


In [131]:
#read electrica & dataframe it
eli_df = pd.read_excel(eli_source,header=4)
eli_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9881 entries, 0 to 9880
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              9881 non-null   object
 1   CORREO ELECTRÓNICO  9611 non-null   object
 2   RUT                 9881 non-null   object
 3   DIRECCIÓN           9881 non-null   object
 4   TELÉFONO            4465 non-null   object
 5   CELULAR             9614 non-null   object
 6   LICENCIA CLASE      9881 non-null   object
dtypes: object(7)
memory usage: 540.5+ KB


In [132]:
#joint gas + eli
joinAll = pd.concat([eli_df,gas_df])
joinAll.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11779 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NOMBRE              11779 non-null  object
 1   CORREO ELECTRÓNICO  11478 non-null  object
 2   RUT                 11779 non-null  object
 3   DIRECCIÓN           11779 non-null  object
 4   TELÉFONO            5367 non-null   object
 5   CELULAR             11458 non-null  object
 6   LICENCIA CLASE      11779 non-null  object
dtypes: object(7)
memory usage: 736.2+ KB


#Definig Transform Functions

##Machine Learning functions

In [153]:
from numpy.lib.function_base import iterable
from pandas.io.formats.style_render import non_reducing_slice
from pandas.core.arrays import boolean
#name normalizer
from typing import Union, List
from enum import Enum
import numpy as np


#IMPORTANT : constant used in keras model version 2
CHARS:List[str] = list('aábcdeéfghiíjklmnñoópqrstuúvwxyz')
NAME_BLOCK_SIZE = 20

class Gender(Enum):
  MALE = 'male'
  FEMALE='female'

#normalize name for keras neuronal net consumption
def nameSerializer(unitName:str,size:int=20)->Union[List[float],None]:
  if unitName == None:
    return None

  name_array = list(unitName)
  # [f,i,r,s,t] ->[6/23,8/23,9/23,11/23,20/23] indexation
  descriteName = list(map(lambda char:CHARS.index(char)/len(CHARS),name_array))
  # content into a 20-size box [0.0 ,0.0 ,....0.0]
  box = [0.0]*(size-len(descriteName))
  input = np.concatenate((np.array(box),np.array(descriteName)))
  return list(input.flatten())


def nameNormalizer(input:str)->list[str]:
  cleaned = input.replace('-',' ')
  splited_name:list[str] = cleaned.lower().split(' ')
  splited_name = list(filter(lambda it: len(it)>1,splited_name))
  return splited_name

#import ML keras tensorflow model
import keras
from keras.optimizers import get
MODEL = 'keras_genderizer_model_v2.h5'
MODEL_PATH = f'/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/{MODEL}'
model = keras.models.load_model(MODEL_PATH)

#wrapping gender
def getGenderByKeras(unitName:str)->Gender|None:
  if unitName == None:
    return None

  options=[Gender.MALE,Gender.FEMALE]
  input = nameSerializer(unitName,20)

  prediction = model.predict([input])[0]
  category = options[np.argmax(prediction)]
  return category

#for saving process simplified selector with just "o" or "a"
def getGenderSimplified(unitName:str)->Gender|None:

  last_char = unitName[-1]
  if last_char == 'o':
    return Gender.MALE
  if last_char == 'a':
    return Gender.FEMALE
  return None

from pandas import DataFrame
WRONG_POSITIVE:DataFrame = pd.read_excel('/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/wrong_positive.xlsx')

def checkWrongPositive(unitName:str|None)-> bool:
  if not unitName:
    return False

  return WRONG_POSITIVE['nombre'].str.contains(unitName).any()

#main general function wrapper⭐
def getGender(name:str | None)->Gender| str | None:

  #checking input is empty
  if name ==  None:
    return None

  #catching wrong positive
  for word in nameNormalizer(name):
    if getGenderSimplified(word) == Gender.MALE:
      return Gender.MALE.value
      break
    if checkWrongPositive(word) == True:
      return Gender.MALE.value
      break

  #if simplified dosen´t work applys de keras model
  modeler  = getGenderByKeras(nameNormalizer(name)[0])
  if modeler:
    return modeler.value

  return None

In [159]:
#testing machine learning
getGender('Carlos - Campos')


1/1 [==============================] - 1s 580ms/step


'male'

#Transform Dataset & calculating columns

In [161]:
# Transform
## split NOMBRE column
joinAll[['surname','name']] = joinAll['NOMBRE'].str.split(',',1,expand=True)
## remove null names
joinAll['name'] = joinAll['name'].map(lambda name: name.strip())
joinAll['address'] = joinAll['DIRECCIÓN'].str.replace('Nunoa','Ñuñoa')

joinAll['movil'] = joinAll['CELULAR']
joinAll['movil'] = joinAll['movil'].str.replace(' ','')
joinAll['movil'] = joinAll['movil'].str.replace('-','')
joinAll['movil'] = pd.to_numeric(joinAll['movil'], errors='coerce').fillna(0).astype(int)
joinAll['movil'] = joinAll['movil'].apply(lambda c:normalizePhone(c))

joinAll['phone'] = joinAll['TELÉFONO']
joinAll['phone'] = joinAll['phone'].str.replace(' ','')
joinAll['phone'] = joinAll['phone'].str.replace('-','')
joinAll['phone'] = pd.to_numeric(joinAll['phone'], errors='coerce').fillna(0).astype(int)


joinAll.head(3)

<ipython-input-161-dedddc9d5402>:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  joinAll[['surname','name']] = joinAll['NOMBRE'].str.split(',',1,expand=True)


,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,surname,name,address,movil,phone
0,"Moraga Núñez , Jorge Orlando",jmoragan@gmail.com,7324296-7,"Cerro Altar 6616, 601, Las Condes",25774233,73381253,A,Moraga Núñez,Jorge Orlando,"Cerro Altar 6616, 601, Las Condes",5.697338e+10,25774233
1,"Sánchez Jorquera , Carlos Mauricio",carlos.msanchezj@gmail.com,18328350-2,"colina 4915, Conchalí",NaN,56940447001,A,Sánchez Jorquera,Carlos Mauricio,"colina 4915, Conchalí",5.694045e+10,0
2,"San Martín Salamanca , Felipe Andrés",felipe.sanmartin.salamanca@gmail.com,18101438-5,"Eleuterio Ramirez 1241, Pobl. 2 de Septiembre,...",NaN,934113388,B,San Martín Salamanca,Felipe Andrés,"Eleuterio Ramirez 1241, Pobl. 2 de Septiembre,...",5.693411e+10,0


##Detecting name gender

In [162]:
joinAll['gender'] = joinAll['name'].map(getGender)

1/1 [==============================] - 0s 82ms/step


In [163]:
#big filter
from pandas import DataFrame
#result:DataFrame = all_technicians.loc[all_technicians['gender']=='female']
result:DataFrame = joinAll
result.sample(10)

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,surname,name,address,movil,phone,gender
7516,"Pavez Roa , Cristian Omar",cpavezroa@gmail.com,15254039-6,"Presidente Rios 771, Villarrica",NaN,59814616,B,Pavez Roa,Cristian Omar,"Presidente Rios 771, Villarrica",5.695981e+10,0,male
1499,"Briones Marchant , Raúl Francisco",rbriones18@hotmail.com,10901464-8,"camino san jose de maipo 05879 05879, Puente Alto",NaN,956541423,3,Briones Marchant,Raúl Francisco,"camino san jose de maipo 05879 05879, Puente Alto",5.695654e+10,0,male
1300,"Gamboa Arellano , Manuel Eusebio",m_gamboa_a@hotmail.com,11830448-9,"los alamos 1556, pobl. vicente martinez, ex-Vi...",NaN,68692449,D,Gamboa Arellano,Manuel Eusebio,"los alamos 1556, pobl. vicente martinez, ex-Vi...",5.696869e+10,0,male
496,"Adriazola Góngora , Juan Carlos",jca.contratista@gmail.com,8451101-3,"ALMIRANTE SIMPSON 1706, 2DO NIVEL, SECTOR ALTO...",245634,65092519,D,Adriazola Góngora,Juan Carlos,"ALMIRANTE SIMPSON 1706, 2DO NIVEL, SECTOR ALTO...",5.696509e+10,245634,male
811,"Bustos Vega , Sergio Mauricio",sbustos@emelectrica.cl,12792679-4,"San Ignacio 999, 77-H, Santiago",25011788,NaN,B,Bustos Vega,Sergio Mauricio,"San Ignacio 999, 77-H, Santiago",NaN,25011788,male
9345,"Zamorano Becerra , José Miguel",josemzb26@gmail.com,17386503-1,"almirante riveros 0630, 404, torre 11, San Ber...",962980196,NaN,B,Zamorano Becerra,José Miguel,"almirante riveros 0630, 404, torre 11, San Ber...",NaN,962980196,male
4187,"AVILA HEREDIA, CRISTIAN ALEJANDRO",cristian.avilah@gmail.com,14556837-4,"Isla Baleares 1715, Pobl.Galicia III, Temuco",251738,(09)68446512,A,AVILA HEREDIA,CRISTIAN ALEJANDRO,"Isla Baleares 1715, Pobl.Galicia III, Temuco",NaN,251738,male
8647,"Vásquez Muñoz , Víctor Manuel",victornd47@gmail.com,17223370-8,"av colon 8396, 44, lan c, Hualpén",51089655,51089655,B,Vásquez Muñoz,Víctor Manuel,"av colon 8396, 44, lan c, Hualpén",5.695109e+10,51089655,male
255,"Soto Millalén , Rodrigo Antonio",rodrigosoto2414@gmail.com,19391813-1,"Puritama sur 01075, 302, Los Acacios, Rancagua",722487571,948757129,3,Soto Millalén,Rodrigo Antonio,"Puritama sur 01075, 302, Los Acacios, Rancagua",5.694876e+10,722487571,male
2999,"ZUÑIGA GONZALEZ, CRISTIAN EDUARDO",crstnzuniga@gmail.com,16195415-2,"iquique 135, cerro placilla, San Antonio",NaN,997777976,B,ZUÑIGA GONZALEZ,CRISTIAN EDUARDO,"iquique 135, cerro placilla, San Antonio",5.699778e+10,0,male


## territory functions

In [164]:
#transform functions

#extract city
def extract_last_col(dir:str)->str|None:
  dir_list = list(reversed(dir.split(',')))
  if len(dir_list)==0:
    return None
  return dir_list[0]


#name case format
import re

def namecase(string:str)->str:
  s_list = re.split('\s+',string.strip())
  for index, word in enumerate(s_list):
    s_list[index] = word[0].upper() + word[1:].lower()
  return ' '.join(s_list)

def setType(input_type:str)->str:
  if input_type == '1 y 4':
    return 'gas'
  if input_type.isnumeric():
    return 'gas'
  else:
    return 'eli'

def getAge(rut:str)->int:
  PENDANT = 3.3363697569700348e-06
  INTERCEPT = 1932.2573852507373
  rut_compose:list[str] = rut.split('-')
  return int(int(rut_compose[0])*PENDANT+INTERCEPT)


  return 0

#Get Territory
from difflib import SequenceMatcher
DPA_PATH = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/CUT2018.xls'
DPA = pd.read_excel(DPA_PATH)
from pandas import Series

class Territory(Enum):
  CITY='city'
  PROVINCE='province'
  REGION='region'

def getTerritory (query:str,req:Territory=Territory.CITY,res:Territory=Territory.REGION)->Series:
  #generate a serie with ratio similatities value
  ratio_series:DataFrame = DPA[req.value].apply(lambda req: SequenceMatcher(None,query,req).ratio())
  #find the most simliar ratio an get this index
  most_similar_index = ratio_series.idxmax()
  #extract row index
  result:Series = DPA.iloc[most_similar_index]
  #return value require, like city, region or province
  return result[res.value]

import math
def normalizePhone(input:int|None)->int|None:

  if len(str(input)) == len('56948611376'):
    return input
  if len(str(input)) == len('948611376'):
    return int('56'+str(input))
  if len(str(input)) == len('48611376'):
    return int('569'+str(input))
  return None

In [165]:
getTerritory('Quillomta',res=Territory.REGION)

'Valparaíso'

In [166]:
#Rename columns
result['city'] = result['address'].map(extract_last_col)
result['city'] = result['city'].apply(lambda c:getTerritory(c,res=Territory.CITY))
result['region'] = result['city'].apply(lambda c:getTerritory(c,res=Territory.REGION))
result['birth'] = result['RUT'].apply(getAge)
result['email'] = result['CORREO ELECTRÓNICO'].str.lower()

#set licence class
result['service'] = result['LICENCIA CLASE'].map(setType)


result.sample(10)

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,surname,name,address,movil,phone,gender,city,region,birth,email,service
7534,"Hurtado Hurtado , Felipe Gabriel",felipe.hurtado.h@gmail.com,17294463-9,"Los maquis 374, Villa las palmera IV, San Bern...",NaN,985221459,A,Hurtado Hurtado,Felipe Gabriel,"Los maquis 374, Villa las palmera IV, San Bern...",5.698522e+10,0,male,San Bernardo,Metropolitana de Santiago,1989,felipe.hurtado.h@gmail.com,eli
722,"Fernández Díaz , Simón Jorge",simonfernandezd@gmail.com,4784014-7,"Av. Hermanos Carrera Oriente 1015, Barrio Mira...",034536348,97780647,3,Fernández Díaz,Simón Jorge,"Av. Hermanos Carrera Oriente 1015, Barrio Mira...",5.699778e+10,34536348,male,San Felipe,Valparaíso,1948,simonfernandezd@gmail.com,gas
843,"Tapia Tapia , Gilberto Hernán",director@inacoch.cl,7368014-K,"Antupiren 7731, J, Penalolén",222788960,93358566,3,Tapia Tapia,Gilberto Hernán,"Antupiren 7731, J, Penalolén",5.699336e+10,222788960,male,Peñalolén,Metropolitana de Santiago,1956,director@inacoch.cl,gas
3999,"Valdevenito Pacheco , Braulio Ariel",VALDEVENITOP@HOTMAIL.COM,12136898-6,"emiliano figueroa 8186, San Ramón",998433756,988099149,D,Valdevenito Pacheco,Braulio Ariel,"emiliano figueroa 8186, San Ramón",5.698810e+10,998433756,male,San Ramón,Metropolitana de Santiago,1972,valdevenitop@hotmail.com,eli
6034,"Marchant Martínez , Fabián Nicolás",fabian.marchant.m@gmail.com,19773956-8,"Pasaje las lavandas 960, Placilla, Valparaíso",NaN,983049917,B,Marchant Martínez,Fabián Nicolás,"Pasaje las lavandas 960, Placilla, Valparaíso",5.698305e+10,0,male,Valparaíso,Valparaíso,1998,fabian.marchant.m@gmail.com,eli
942,"CASTRO ARDILES , Oscar ANDRÉS ANTONIO",oscar.castro.a@gmail.com,16057346-5,"San Martn 682, ex-Villa Alemana",NaN,987805516,A,CASTRO ARDILES,Oscar ANDRÉS ANTONIO,"San Martn 682, ex-Villa Alemana",5.698781e+10,0,male,Villa Alemana,Valparaíso,1985,oscar.castro.a@gmail.com,eli
805,"Vargas Soto , Víctor Manuel",vicvarm@gmail.com,8649660-7,"Aucar SN, sector Aucar rural Quemchi, Qu...",NaN,995172403,D,Vargas Soto,Víctor Manuel,"Aucar SN, sector Aucar rural Quemchi, Qu...",5.699517e+10,0,male,Quemchi,Los Lagos,1961,vicvarm@gmail.com,eli
6244,"Cortés Farías , Víctor Antonio",vcortesfarias@gmail.com,14906622-5,"amador neghme 1131, 203, block d altos del sol...",NaN,991330208,B,Cortés Farías,Víctor Antonio,"amador neghme 1131, 203, block d altos del sol...",5.699133e+10,0,male,Arica,Arica y Parinacota,1981,vcortesfarias@gmail.com,eli
7547,"Zúñiga Mundaca , Juan Bernardo",juan.zga86@gmail.com,16396041-9,"bio bio 5, pasaje rio duqueco, Chiguayante",413180370,86852209,B,Zúñiga Mundaca,Juan Bernardo,"bio bio 5, pasaje rio duqueco, Chiguayante",5.698685e+10,413180370,male,Chiguayante,Biobío,1986,juan.zga86@gmail.com,eli
7654,"Prado Marambio , Miguel Antonio",mpradomarambio@gmail.com,14631831-2,"Mendez 736, Antofagasta",52205746,87619635,A,Prado Marambio,Miguel Antonio,"Mendez 736, Antofagasta",5.698762e+10,52205746,male,Antofagasta,Antofagasta,1981,mpradomarambio@gmail.com,eli


In [167]:
#content with Name Format
for field in ['name','surname','address','city']:
  result[field] = result[field].map(namecase)

result.sample(10)

,NOMBRE,CORREO ELECTRÓNICO,RUT,DIRECCIÓN,TELÉFONO,CELULAR,LICENCIA CLASE,surname,name,address,movil,phone,gender,city,region,birth,email,service
4676,"Pérez Pizarro , Pedro Pablo",pedro.perez@chileenergy.cl,13979131-2,"Los Espinos 2597, Larapinta, Lampa",978079563,978079563,B,Pérez Pizarro,Pedro Pablo,"Los Espinos 2597, Larapinta, Lampa",5.697808e+10,978079563,male,Lampa,Metropolitana de Santiago,1978,pedro.perez@chileenergy.cl,eli
1064,"Alvarado Astorga , Renato Domingo Adrián",servicioatodogas@gmail.com,7250219-1,"ANGOSTURA INGLESA 248, Las Condes",NaN,84028804,3,Alvarado Astorga,Renato Domingo Adrián,"Angostura Inglesa 248, Las Condes",5.698403e+10,0,male,Las Condes,Metropolitana de Santiago,1956,servicioatodogas@gmail.com,gas
2880,"Bravo González , Iván Rodrigo",ivanrodrigobg@gmail.com,13019350-1,"avenida Parque Sur 18731, Maipú",978576610,56978576610,B,Bravo González,Iván Rodrigo,"Avenida Parque Sur 18731, Maipú",5.697858e+10,978576610,male,Maipú,Metropolitana de Santiago,1975,ivanrodrigobg@gmail.com,eli
84,"Currin Quelempán , Raúl Ronald Alexander",ronaldcurrin.q@gmail.com,15987172-K,"Calama 326, Temuco",NaN,967595603,1,Currin Quelempán,Raúl Ronald Alexander,"Calama 326, Temuco",5.696760e+10,0,male,Temuco,La Araucanía,1985,ronaldcurrin.q@gmail.com,gas
2357,"Rocco Aros , Pedro Andrés",pedroroccoa@gmail.com,15713478-7,"Parinacota 1869, Casa, Llo lleo, San Antonio",979305790,979305790,D,Rocco Aros,Pedro Andrés,"Parinacota 1869, Casa, Llo Lleo, San Antonio",5.697931e+10,979305790,male,San Antonio,Valparaíso,1984,pedroroccoa@gmail.com,eli
6866,"García García , José Alfredo",jose_garcia@live.cl,17217350-0,"Pje apolo 2251, Villa MonteCarlo, Penaflor",NaN,71289423,B,García García,José Alfredo,"Pje Apolo 2251, Villa Montecarlo, Penaflor",5.697129e+10,0,male,Peñaflor,Metropolitana de Santiago,1989,jose_garcia@live.cl,eli
8617,"Millar Arriagada , Héctor Manuel",millar.hector@gmail.com,19082895-6,"AVENIDA CHILE 476, Chillan",NaN,941706097,A,Millar Arriagada,Héctor Manuel,"Avenida Chile 476, Chillan",5.694171e+10,0,male,Chillán,Ñuble,1995,millar.hector@gmail.com,eli
8403,"Cornejo Leiva , Cristian Osvaldo",ccornejo666@gmail.com,12014558-4,"cinco 0475, poblacion argentina paso hondo qui...",NaN,91814372,A,Cornejo Leiva,Cristian Osvaldo,"Cinco 0475, Poblacion Argentina Paso Hondo Qui...",5.699181e+10,0,male,Quilpué,Valparaíso,1972,ccornejo666@gmail.com,eli
157,"Marambio Pirul , Daniel Alejandro",DANIEL.MARAMBIO@HOTMAIL.COM,15433189-1,"parque boscoso 2315 puente alto 2315, Puente Alto",957193385,57193385,A,Marambio Pirul,Daniel Alejandro,"Parque Boscoso 2315 Puente Alto 2315, Puente Alto",5.695719e+10,957193385,male,Puente Alto,Metropolitana de Santiago,1983,daniel.marambio@hotmail.com,eli
4375,"Villagrán Ibarra , Daniel Sebastián",daniel.villagrani@gmail.com,15360126-7,"Rovretto 1036, F 64610958, Coquimbo",964610958,964610958,A,Villagrán Ibarra,Daniel Sebastián,"Rovretto 1036, F 64610958, Coquimbo",5.696461e+10,964610958,male,Coquimbo,Coquimbo,1983,daniel.villagrani@gmail.com,eli


In [169]:
#remove unnecesary data
result = result.drop(['NOMBRE','CORREO ELECTRÓNICO','DIRECCIÓN','CELULAR','TELÉFONO'],axis=1)


In [170]:
result = result.rename(columns ={'LICENCIA CLASE':'licence','RUT':'rut'})
result.sample(10)

,rut,licence,surname,name,address,movil,phone,gender,city,region,birth,email,service
166,18778011-K,C,Huerta Álvarez,Angelo Nicolás,"Av La Florida 10269, 416, Torre B, La Florida",5.699664e+10,0,male,La Florida,Metropolitana de Santiago,1994,huerta.angelone@gmail.com,eli
8975,9485961-1,A,Jeraldino González,Jorge Antonio,"Manuel Rodriguez 104-a1, Buin",5.699041e+10,0,male,Buin,Metropolitana de Santiago,1963,jorge.jer@hotmail.cl,eli
5951,15864728-1,A,Barra Bascur,Marcos Danilo,"Pasaje 3 2047, Población Ampliación Pablo Neru...",5.699731e+10,435689,male,Coyhaique,Aysén del General Carlos Ibáñez del Campo,1985,marcosbarrabascur@gmail.com,eli
3809,15525476-9,B,Letelier Medina,Mauricio Edgardo,"Pedro De Miranda, Lo Miranda 964, Centro, Doñihue",5.698748e+10,466115,male,Doñihue,Libertador General Bernardo O'Higgins,1984,urbaelec@gmail.com,eli
8549,15848016-6,D,Mansilla Villarroel,Alexis Marcelo,"Dolores Pincheira 691, Villa Los Poetas, Puert...",5.697397e+10,0,male,Puerto Montt,Los Lagos,1985,21.alexis@gmail.com,eli
9276,11433017-5,C,Mansilla Muñoz,José Baudilio,"Bdo.phillipe 942, Villa Los Colonos, Castro",5.699920e+10,0,male,Castro,Los Lagos,1970,petermansilla@gmail.com,eli
5684,16000702-8,B,Ortiz Madrid,Jonathan Alejandro,"Belgrano 896, Quilpué",5.698799e+10,0,male,Quilpué,Valparaíso,1985,NaN,eli
7047,17977790-8,A,Rojas Barboza,Eduardo Ignacio,"11 De Noviembre 663, Quillota",5.699548e+10,332517178,male,Quillota,Valparaíso,1992,eduardo.rojas.barboza@gmail.com,eli
990,17542530-6,C,Cerda Herrera,José Daniel,"Mac-iver 0830, Ampliacion Sur, Chañaral",5.698735e+10,0,male,Chañaral,Atacama,1990,josecerda_1992@hotmail.com,eli
976,9119695-6,A,Yz Oyaneder,Jaime Gonzalo,"Almendral 4865, Ñuñoa",5.695829e+10,0,male,Ñuñoa,Metropolitana de Santiago,1962,jaime.yz@hunno.cl,eli


In [171]:
result_path = f'{PATH_GDRIVE}/sec_result.xlsx'
result.to_excel(result_path)

# Optional filter

In [ ]:
reload_path = f'{PATH_GDRIVE}/sec_result.xlsx'
reloaded = pd.read_excel(reload_path)

Filter regional (*optional*)

In [ ]:
result_path_v = f'{PATH_GDRIVE}/sec_result_v.xlsx'
reloaded.loc[reload['region']=='Valparaíso'].to_excel(result_path_v)

In [ ]:
reloaded.sample(5)

,Unnamed: 0,email,rut,address,phone,licence,Surname,names,gender,movil,city,region,birth,service
3346,3346,emauricio.rojas@gmail.com,16737596-0,"Las Palmas 65, Villa San Judas Tadeo, Malloa",976053172,B,Rojas Mañán,Edwin Mauricio,male,5.697605e+10,Malloa,Libertador General Bernardo O'Higgins,1988,eli
9418,9418,patriciomolina019@gmail.com,18207270-2,"Ramirez 2297, Villa Abedules, La Union",967615944,A,Molina Reyes,Patricio Antonio,male,5.696762e+10,La Unión,Los Ríos,1993,eli
1971,1971,moi.solar29@gmail.com,18326150-9,"Cerro Nevado 787, Quilicura",NaN,D,Solar Zúñiga,Moisés Alejandro,male,5.693011e+10,Quilicura,Metropolitana de Santiago,1993,eli
9621,9621,bonifaciocuri@gmail.com,16464975-K,"Poeta Juan Luis Martínez 1758, Santa María De ...",NaN,B,Maldonado Maldonado,Denis Marcelo,male,5.699843e+10,Colina,Metropolitana de Santiago,1987,eli
409,409,riego.agric@gmail.com,10791345-9,"Esteban Grlica 632, Villa Las Salinas Sindempa...",051289194,B,Rojas Cisternas,Jorge Wilson,male,5.696850e+10,Coquimbo,Coquimbo,1968,eli


In [ ]:
reloaded.to_excel(reload_path)

#Bulk load to firebase

In [173]:
#firebase
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
#service as account
credential_path='/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/firebase.json'
credential = credentials.Certificate(credential_path)
firebase_admin.initialize_app(credential)
db = firestore.client()

ValueError: ignored

In [174]:
#sec_result reload
source_path = '/content/drive/MyDrive/MinEnergia/99.Data/GenderEquality/sec_result.xlsx'
source = pd.read_excel(source_path)
source.sample(8)


,Unnamed: 0,rut,licence,surname,name,address,movil,phone,gender,city,region,birth,email,service
8082,8082,18696791-7,A,Valladares Chacón,Benjamín Alejandro,"Leucotón 0877, La Pintana",5.695654e+10,0,male,La Pintana,Metropolitana de Santiago,1994,valladares.ch@gmail.com,eli
8918,8918,13844169-5,A,Castro Grandón,Cristian David,"Zaragoza 229, Valle Noble, Concepción",5.698882e+10,459501,male,Concepción,Biobío,1978,cristiandcg@gmail.com,eli
6184,6184,12839504-0,B,Salfate Codoceo,Henry David,"Atacama 898, Taltal",5.698460e+10,56222794594,male,Taltal,Antofagasta,1975,hsalfatec@gmail.com,eli
11148,1267,13660439-2,1,Urbina Meneses,Hugo Alejandro,"Santo Domingo 2810, Santiago",5.695103e+10,0,male,Santiago,Metropolitana de Santiago,1977,urbina.ten@gmail.com,gas
5731,5731,15986837-0,B,Matus Ríos,Andrés José,"Los Pimientos 680, Rafael Carballo, Graneros",5.698371e+10,0,male,Graneros,Libertador General Bernardo O'Higgins,1985,chispa_matus@hotmail.com,eli
10305,424,16714569-8,3,Campos Cristi,Pablo Esteban,"Av Circunvalación Norte 01312, 501, Pinares, R...",5.697709e+10,0,male,Rancagua,Libertador General Bernardo O'Higgins,1988,pablocampos.pcc@gmail.com,gas
8394,8394,14135934-7,A,Bravo Moraga,Luis Alfredo,"Av. Ossa 347, 1607-a, La Cisterna",5.697497e+10,74970630,male,La Cisterna,Metropolitana de Santiago,1979,bmluis@gmail.com,eli
10933,1052,14532338-K,2,Pastén Solís,Fredi Wilson,"Venezuela 2825, Calama",5.696372e+10,0,male,Calama,Antofagasta,1980,fps@vtr.net,gas


In [182]:
source = source.replace({np.nan:None})
source['phone'] = source['phone'].replace({0:None})
source.sample(5)

,Unnamed: 0,rut,licence,surname,name,address,movil,phone,gender,city,region,birth,email,service
7173,7173,15454136-5,A,Valdés Muñoz,Julio Ernesto,"Ernesto Cea 1196, Villa San Martin, Cerrillos",56989747561.0,25383233,male,Cerrillos,Metropolitana de Santiago,1983,juliot52@gmail.com,eli
8445,8445,16575521-9,A,Castro Garcia,Javier Osvaldo,"Rodriguez 308, Ex-limache",56987674905.0,None,male,Limache,Valparaíso,1987,jav.castro.garcia@gmail.com,eli
11106,1225,10279761-2,1,Torres Cofré,Boris Guillermo,"Avenida La Marina 1270, 202, San Miguel",56993212118.0,229805256,male,San Miguel,Metropolitana de Santiago,1966,torresbor@yahoo.es,gas
5491,5491,9579194-8,A,Cruz Soto,Luis Gabriel,"La Concepcion 81, 707, Providencia",56995376783.0,None,male,Providencia,Metropolitana de Santiago,1964,lcs.ingespro@gmail.com,eli
9680,9680,10605921-7,A,Hernández Gallardo,Carlos Alejandro,"El Avellano 3873, Villa La Arboleda De Macul, ...",56994386725.0,None,male,Macul,Metropolitana de Santiago,1967,carlos.chernandez@gmail.com,eli


In [183]:
#database references
providers_collection = db.collection('energy_providers').document('source_providers').collection('providers')

In [184]:
from pytz import timezone
from traitlets.traitlets import Dict
import datetime
#document converter
def provider_converter(it):
  timestamp=datetime.datetime.now(tz=timezone('America/Santiago'))

  return {
      "rut":it.rut,
      "personal":{"name":it.name,"surname":it.surname,"birth":it.birth,"gender":it.gender},
      "licence":{"category":it.licence,"service":it.service},
      "address":{"street":it.address,"city":it.city,"region":it.region},
      "contact":{
          "phone": int(it.phone) if it.phone!=None else None ,
          "movil":int(it.movil) if it.movil!=None else None ,
          "email":it.email
          },
      "update_at":timestamp,
  }



Bulk upload, test 2 times, and test again, and after go in peace

In [ ]:
from google.cloud.firestore_v1.base_query import FieldFilter
#load
sized = source
for tech in sized.itertuples():
  #fetch if exists
  docs = providers_collection.where(filter=FieldFilter('rut','==',tech.rut)).stream()
  docs_list = list(docs)
  #if exist 1 update
  if len(docs_list) == 1:
    put_doc = docs_list[0]
    providers_collection.document(put_doc.id).set(provider_converter(tech),merge=True)
    print('updated',put_doc.id)
  #if not with 0 create
  if len(docs_list) == 0:
    post_doc = providers_collection.document()
    post_doc.set(provider_converter(tech),merge=True)
    print('created',post_doc.id)




